In [0]:
spark.sql("USE CATALOG movielens")
spark.sql("USE SCHEMA bronze")

catalog = "movielens"
bronze  = "movielens.bronze"
silver  = "movielens.silver"

## Movies

In [0]:
from pyspark.sql import functions as F, Window as W

movies_bronze = spark.table(f"{bronze}.movie")

empty_arr = F.expr("array()")

movies_silver = (
    movies_bronze
      .withColumn("movieId", F.col("movieId").cast("bigint"))
      .withColumn("title",   F.trim("title"))
      .withColumn("year_str", F.regexp_extract(F.col("title"), r"(19|20)\d{2}", 0))
      .withColumn("year", F.when(F.col("year_str") == "", None)
                           .otherwise(F.col("year_str").cast("int")))
      .withColumn("genres_raw", F.when(
          (F.col("genres").isNull()) | (F.col("genres") == "(no genres listed)"),
          None
      ).otherwise(F.col("genres")))
      .withColumn("genres", F.when(
          (F.col("genres").isNull()) | (F.col("genres") == "(no genres listed)"),
          empty_arr
      ).otherwise(F.split(F.col("genres"), r"\|")))
      .select("movieId", "title", "year", "genres_raw", "genres")
)

(movies_silver.write
    .format("delta").mode("overwrite")
    .saveAsTable(f"{silver}.movies"))

movie_genres_silver = (
    movies_silver
      .withColumn("genre", F.explode("genres"))
      .select("movieId", F.trim("genre").alias("genre"))
      .where(F.col("genre") != "")
)

(movie_genres_silver.write
    .format("delta").mode("overwrite")
    .saveAsTable(f"{silver}.movie_genres"))

## Ratings

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW _ratings_dedup AS
SELECT *,
       ROW_NUMBER() OVER (
         PARTITION BY userId, movieId, timestamp, rating
         ORDER BY timestamp DESC
       ) AS rn
FROM {bronze}.rating
WHERE rating BETWEEN 0.5 AND 5.0
""")

spark.sql(f"""
CREATE OR REPLACE TABLE {silver}.ratings
USING delta AS
SELECT
  CAST(userId AS BIGINT)   AS userId,
  CAST(movieId AS BIGINT)  AS movieId,
  CAST(rating AS DOUBLE)   AS rating,
  timestamp                AS rating_ts,
  TO_DATE(timestamp)       AS rating_date
FROM _ratings_dedup
WHERE rn = 1
""")

spark.sql(f"""
CREATE OR REPLACE VIEW {silver}.ratings_latest AS
SELECT userId, movieId, rating, rating_ts, rating_date
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY userId, movieId ORDER BY rating_ts DESC) AS rn
  FROM {silver}.ratings
)
WHERE rn = 1
""")

## Tags

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW _tags_clean AS
SELECT
  CAST(userId AS BIGINT)  AS userId,
  CAST(movieId AS BIGINT) AS movieId,
  TRIM(tag)               AS tag,
  timestamp               AS tag_ts
FROM {bronze}.tag
""")

spark.sql(f"""
CREATE OR REPLACE TEMP VIEW _tags_dedup AS
SELECT *,
       ROW_NUMBER() OVER (
         PARTITION BY userId, movieId, tag, tag_ts
         ORDER BY tag_ts DESC
       ) AS rn
FROM _tags_clean
WHERE tag IS NOT NULL AND tag <> ''
""")

spark.sql(f"""
CREATE OR REPLACE TABLE {silver}.tags
USING delta AS
SELECT userId, movieId, tag, tag_ts, TO_DATE(tag_ts) AS tag_date
FROM _tags_dedup
WHERE rn = 1
""")

## Links

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {silver}.links
USING delta AS
SELECT
  CAST(movieId AS BIGINT) AS movieId,
  CAST(imdbId  AS BIGINT) AS imdbId,
  CAST(tmdbId  AS BIGINT) AS tmdbId
FROM {bronze}.link
""")

## Genome Tags

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {silver}.genome_tags
USING delta AS
SELECT
  CAST(tagId AS BIGINT) AS tagId,
  TRIM(tag)             AS tag
FROM {bronze}.genome_tags
WHERE tag IS NOT NULL AND TRIM(tag) <> ''
""")

## Genome Score

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {silver}.genome_scores
USING delta AS
SELECT
  CAST(s.movieId AS BIGINT)  AS movieId,
  CAST(s.tagId   AS BIGINT)  AS tagId,
  CAST(s.relevance AS DOUBLE) AS relevance
FROM {bronze}.genome_scores s
JOIN {silver}.genome_tags t USING (tagId)
WHERE s.relevance BETWEEN 0.0 AND 1.0
""")